## Imports

In [ ]:
# Only needs to be run once

"""!pip install pycm""";

In [ ]:
import boto3
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
from pycm                    import ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.metrics         import balanced_accuracy_score
from sklearn.metrics         import classification_report
from keras                   import regularizers
from keras.models            import Sequential
from keras.layers            import Dense, Flatten, Dropout
from keras.layers            import Conv2D, MaxPooling2D
from keras.utils             import np_utils
from IPython.core.display    import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

# Declaring a random state for the modeling process

np.random.seed(42)

## Table Contents
     
 
- [Reading In The New Data](#Reading-In-The-New-Data)
    - [Data Overview](#Data-Overview)
    - [Mapping Labels](#Mapping-Labels)
    
    
- [Preprocessing](#Preprocessing)
    - [Declaring X and y](#Declaring-X-and-y)    
    - [Train-Test Splitting The Data](#Train-Test-Splitting-The-Data)
    - [Reshaping The Data](#Reshaping-The-Data)
    
    
- [Modeling](#Modeling)
    - [Setting Up The Neural Network](#Setting-Up-The-Neural-Network)
    - [Compiling The Neural Network](#Compiling-The-Neural-Network)
    - [Fitting The Training Data](#Fitting-The-Training-Data)
    - [Generating Predictions](#Generating-Predictions)
 
 
- [Evaluation](#Evaluation)

## Reading In The Data

In [ ]:
# Reading in my credentials

with open("access_key", "r") as key:
    access_key = key.read()
    
with open("secret_key", "r") as secret:
    secret_access_key = secret.read()
    
# Declaring the client as an s3 bucket
    
client = boto3.client('s3', 
                      aws_access_key_id     = access_key,
                      aws_secret_access_key = secret_access_key)

# Naming the bucket and file

bucket_name = "google-quick-draw"
object_key  = "animals.csv"

# Creating a csv object

csv_obj = client.get_object(Bucket = bucket_name, 
                            Key    = object_key)

body = csv_obj['Body']

# Reading in the data

animals = pd.read_csv(body)

### Data Overview

In [ ]:
# Checking the dimensions of the dataframe

animals.shape

In [ ]:
# Looking at the head five rows

animals.head()

In [ ]:
animals.info()

In [ ]:
# Looking for null values

animals.isnull().sum().sum()

### Mapping Labels

In [ ]:
animals["label"].replace(to_replace = {"bear": 0,
                                       "cat" : 1,
                                       "dog" : 2},
                         inplace    = True)

# Making sure the labels are integers

animals["label"].dtype

## Preprocessing

### Declaring X and y

In [ ]:
# X variable

X = np.array(animals.drop(columns = "label")) 

# y variable

y = animals["label"].values

### Train-Test Splitting The Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state = 42, 
                                                    test_size    = 0.3)

# Keeping copies of the originals

y_train_org = y_train

y_test_org  = y_test

In [ ]:
# Scaling each X value

X_train = X_train.astype(float)
X_train /= 255

X_test  = X_test.astype(float)
X_test  /= 255

In [ ]:
# Making sure the y variables are categorical

y_train = np_utils.to_categorical(y_train, 3)

y_test  = np_utils.to_categorical(y_test, 3)

### Reshaping The Data

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)

X_test  = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
# Checking the data type of the train and test sets

print(f"X_train's data type is {type(X_train)}")

print(f"X_test's data type is {type(X_test)}")

## Modeling

### Setting Up The Neural Network

#### Instantiating The Model

In [ ]:
cnn = Sequential()

#### Adding Layers

In [ ]:
# Convolutional layers

# The first convolutional layer

cnn.add(Conv2D(filters     = 6,
               kernel_size = 3,
               activation  = "relu",
               input_shape = (28,28,1)))

# The first maxpooling layer

cnn.add(MaxPooling2D(pool_size = (2,2)))

# The second convolutional layer

cnn.add(Conv2D(128,
               kernel_size = 3,
               activation  = "relu"))

# The second maxpooling layer

cnn.add(MaxPooling2D(pool_size = (2,2)))


In [ ]:
# Dense layers

# Flattening the data

cnn.add(Flatten())

# Adding the first dense layer

cnn.add(Dense(128,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.0001)))

# Adding first dropout

cnn.add(Dropout(rate = 0.25))

# Adding the second dense layer

cnn.add(Dense(64,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.0001)))
                
# Adding second dropout

cnn.add(Dropout(rate = 0.25))

# Adding the third dense layer

cnn.add(Dense(32,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.0001)))

# Adding second dropout

cnn.add(Dropout(rate = 0.25))

# Adding the output layer: 3 because there are three classes

cnn.add(Dense(3,
              activation = "softmax"))

### Compiling The Neural Network

In [ ]:
cnn.compile(loss      = "categorical_crossentropy",
            optimizer = "adam",
            metrics   = ["acc"])

### Fitting The Training Data

In [ ]:
cnn.fit(X_train,
        y_train,
        validation_data = (X_train, y_train),
        batch_size      = 256,
        epochs          = 25,
        verbose         = 0)

In [ ]:
# Training predictions

train_preds = cnn.predict_classes(X_train,
                                  batch_size = 100,
                                  verbose    = 0).ravel()

# Test predictions

test_preds = cnn.predict_classes(X_test,
                                 batch_size = 100,
                                 verbose    = 0).ravel()

In [ ]:
balanced_accuracy_score(y_train_org, train_preds)

In [ ]:
balanced_accuracy_score(y_test_org, test_preds)

[Top](#Table-Of-Contents)